Predict Biased Word then use Marta's Features
------
We first use the bias detection module and then convert that word into a feature representation using Marta's features.

The features we use in this example are the ones proposed by Recasens et al, 2013 (https://nlp.stanford.edu/pubs/neutrality.pdf). For each word her features generate 32 features. Our labeling function then takes these features and applies a very simple regression (1 layer feed forward neural net).

In [1]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params

%load_ext autoreload
%autoreload 2

In [2]:
params = Params.read_params("experiment_params.json")

In [3]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)

03/15/2020 20:55:30 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 5513.82it/s]


In [4]:
from src.utils.shared_utils import get_bias_predictions 

In [5]:
bias_predictions = get_bias_predictions(dataset, params.intermediary_task, params.dataset, batch_size=8)

03/15/2020 20:55:31 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/15/2020 20:55:31 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/15/2020 20:55:31 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /juice/scr/scr110/scr/nlp/bias_classification/experiments/workshop_experiments/baselines/small_data/predict-recasens/t

In [6]:
dataset.add_data(bias_predictions, "pre_tok_label_ids_predictions")

In [7]:
# importing the Featurizer created by Pryzant et al.
from src.utils.weak_labeling_utils import get_marta_featurizer, extract_marta_features

In [8]:
dataset

Length: 324 Keys: dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label', 'pre_tok_label_ids_predictions'])

In [9]:
featurizer = get_marta_featurizer(params.dataset)

03/15/2020 20:56:10 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [10]:
marta_features = extract_marta_features(dataset, featurizer, bias_key="pre_tok_label_ids_predictions")

In [11]:
dataset.add_data(marta_features, "marta_features")

### This is where the classification experiment starts

In [12]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [13]:
from src.utils.classification_utils import run_bootstrapping

In [14]:
marta_features.shape

torch.Size([324, 90])

In [15]:
statistics = run_bootstrapping(classification_experiment, dataset, params.final_task, num_bootstrap_iters=5, input_key='marta_features', label_key='bias_label', threshold=0.5)

In [16]:
statistics

{'auc': [(0.8576076762547599, 0.9030487839422832), 0.8806886014024201],
 'accuracy': [(0.7878188775510204, 0.8938775510204082), 0.8480867346938776]}